# Data Ingestion

In [ ]:
!wget -q https://www.dropbox.com/s/vs6ocyvpzzncvwh/new_articles.zip

In [ ]:
!unzip -q new_articles.zip -d new_articles

In [ ]:
!pip install langchain langchain-openai langchain-chroma chromadb langchain-community

In [ ]:
!pip show langchain langchain-openai langchain-chroma chromadb langchain-community

In [ ]:
!pip freeze > requirements.txt

In [ ]:
!cat requirements.txt

In [ ]:
from google.colab import files
files.download('requirements.txt')

In [ ]:
# !pip install langchain-community

## Load data

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

In [ ]:
# Load documents
loader = DirectoryLoader("/content/new_articles/", glob="./*.txt", loader_cls=TextLoader)
documents = loader.load()
documents

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
# Split documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
texts = text_splitter.split_documents(documents)
texts

In [ ]:
len(texts)

In [ ]:
texts[0]

In [ ]:
texts[1]

## Creating DB

In [ ]:
# !pip install langchain langchain-openai langchain-chroma chromadb

In [ ]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [ ]:
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma

# 1) Create the embedding model
embeddings = OpenAIEmbeddings(model="text-embedding-3-small", api_key = OPENAI_API_KEY)

# 2) Persist directory for storing the vector DB
persist_directory = "db"

# 3) Create and persist the vector store from documents
vectordb = Chroma(
    collection_name="my_collection",               # required
    embedding_function=embeddings,
    persist_directory=persist_directory            # enables persistence
)


# Add documents to the vector DB
# documents should be a list of Document objects
vectordb.add_documents(documents = texts)

In [ ]:
# Save changes
# vectordb.persist()

# Clear in-memory reference
vectordb = None

In [ ]:
# 4) Load the persisted DB back from disk
vectordb = Chroma(
    collection_name="my_collection",               # same collection name
    embedding_function=embeddings,
    persist_directory=persist_directory
)

## Make a retriever

In [ ]:
# 5) Create a retriever
retriever = vectordb.as_retriever()

# 6) Get relevant docs for a query
docs = retriever.invoke("How much money did Microsoft raise?")

In [ ]:
len(docs)

In [ ]:
docs

In [ ]:
# 7) Example of customizing search parameters
retriever = vectordb.as_retriever(search_kwargs={"k": 2})

# You can inspect these values
print(retriever.search_type)
print(retriever.search_kwargs)

In [ ]:
retriever.search_type

In [ ]:
retriever.search_kwargs

## Make a chain

In [ ]:
# !pip show langchain langchain-openai langchain-chroma chromadb

In [ ]:
# !pip install langchain

In [ ]:
# !pip install langchain-community


In [ ]:
# !pip show langchain langchain-openai langchain-chroma chromadb langchain-community

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

In [ ]:
# 1️) LLM
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, api_key = OPENAI_API_KEY)

# 2️) Prompt template
prompt = ChatPromptTemplate.from_template(
    """
    Answer the question using ONLY the context below.
    If the answer is not in the context, say "I don't know".

    Context:
    {context}

    Question:
    {question}
    """
)

# 3️) LCEL RAG Chain

full_rag_chain = (
    RunnablePassthrough.assign(context=itemgetter("question") | retriever)
    | {
        "answer": prompt | llm | StrOutputParser(),
        "context": itemgetter("context")
    }
)

In [ ]:
# 4) Helper function
def process_llm_response(response):
    print("Answer:\n")
    print(response["answer"])
    print("\nSources:")
    for doc in response["context"]:
        source = doc.metadata.get("source", "Unknown source")
        print(f"- {source}")

In [ ]:
# Full example
query = "How much money did Microsoft raise?"
response = full_rag_chain.invoke({"question": query})
process_llm_response(response)

In [ ]:
# Another query
query = "What is the news about Pando?"
response = full_rag_chain.invoke({"question": query})
process_llm_response(response)

## Deleteing the DB


In [ ]:
!zip -r db.zip ./db

In [ ]:
# 1. Access the underlying client to delete
try:
    vectordb._client.delete_collection("my_collection")
    print("Collection deleted successfully.")
except Exception as e:
    print(f"Collection might not exist: {e}")

# 2. Manual directory cleanup (Optional but recommended for a total reset)
import shutil
import os

db_path = "db"
if os.path.exists(db_path):
    shutil.rmtree(db_path)
    print("Database directory removed.")

## Starting again loading the db

In [ ]:
!unzip db.zip